# The Anatomy of a mewithoutYou Setlist
Exploring mewithoutYou's 2018-2019 [Untitled] Tour

## Standard Analysis Setup

In [ ]:
# Standard imports to run concert_analytics

import sys
import os

PROJECT_ROOT = os.path.abspath(os.path.join(os.path.dirname(""), "../../"))
sys.path.insert(0, PROJECT_ROOT)

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from plotnine import theme, element_text, element_blank, element_rect, element_line

from postgres import fetch_table,run_query

import logging

logging.basicConfig(level=logging.INFO)  # DEBUG < INFO < WARNING < ERROR < CRITICAL

pd.set_option("display.max_columns", None)  # Show all columns

# Step 2: Define the color scale
album_colors = {
    "A To B Life": "#FB9E50",
    "Brother, Sister": "#F986BA",
    "Catch For Us The Foxes": "#3C7DC4",
    "It's All Crazy! It's All False! It's All A Dream! It's Alright": "#D64848",
    "Pale Horses": "#F25C05",
    "Pale Horses: Appendix":"#A7ECF5",
    "Ten Stories": "#5D4E8C",
    "[Untitled]": "#33C27D",
    "[untitled] e.p.": "#F6D357"
}

## Fetch our dataset

In [ ]:
# fetch setlist and spotify metadata

query="""
with track_link_cte as (
	select 
 		sim.event_set_song_id
 		,sim.track_id
 		,row_number() over (partition by sim.event_set_song_id order by similarity_score desc nulls last) as similarity_rank_post_filter
	 from 
	 	analytics_mart.mart_track_setlist_similarity_scores as sim 
	 	join analytics_mart.mart_all_tracks as all_tr
	 		on sim.track_id = all_tr.track_id
	 where true
	 	and all_tr.album_id not in ('4bGIhBIGsziCakPziupVmQ','39P5R1B5XF1dGhpojrukVA','06y12naBReESEe7u2OFG7j')
)
, track_link_filtered_cte as (
	select 
 		*
	 from 
	 	track_link_cte
	 where true
	 	and similarity_rank_post_filter = 1
)
select 
	msh.*
	,tr.*
	,es.event_total_songs
	,es.event_total_sets
	,es.event_total_encore_songs
	,es.event_total_non_encore_songs
from 
	analytics_mart.mart_setlist_history as msh
	left join track_link_filtered_cte as tl_cte
		on msh.event_set_song_id = tl_cte.event_set_song_id
	left join analytics_mart.mart_all_tracks as tr
		on tl_cte.track_id = tr.track_id
    left join analytics_mart.mart_event_summary as es
    	on msh.event_id = es.event_id
where true
    and msh.event_tour_id = 'dff91b2e201d2bfdb4f7a16b57c62194'
    and msh.event_date <= '2019-01-01'
"""

df = run_query(query)
df.to_csv('mewithoutYou_Untitled_Tour_Data.csv')
df.head()

## Preview Color Scheme

In [ ]:
def preview_palette(color_dict):
    fig, ax = plt.subplots(figsize=(8, 1))
    for i, (label, color) in enumerate(color_dict.items()):
        ax.bar(i, 1, color=color)
        ax.text(i, -0.3, label, ha='center', fontsize=10, rotation=45)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlim(-0.5, len(color_dict)-0.5)
    ax.set_title("Custom Album Color Palette")
    plt.show()

preview_palette(album_colors)

## How many songs per night?

In [ ]:
# songs_per_night = 
songs_per_night = (
    df.groupby('event_date')['event_set_song_id']
    .nunique()
    .reset_index(name='unique_song_count')
    .sort_values('event_date')
)

total_shows = songs_per_night['event_date'].nunique()

average_songs = int(songs_per_night['unique_song_count'].mean())

print(f'{total_shows} shows with {average_songs} songs')


from plotnine import ggplot, aes, geom_col, labs, theme, element_text, element_rect, scale_x_datetime
from mizani.breaks import date_breaks
from mizani.formatters import date_format

base_family_axis="Courier New"
base_family_title="Helvetica"

# Build the plot
g = (
    ggplot(songs_per_night, aes(x='event_date', y='unique_song_count')) +
    geom_col(fill='#33C27D') +
    scale_x_datetime(date_labels="%b %d", breaks=date_breaks(width="2 week")) +
    labs(
        title=f"mewithoutYou played {total_shows} shows throughout \ntheir tour, playing {average_songs} songs on the average night",
        subtitle="Unique songs played by show in the 2018 [Untitled] tour",
        y="Unique Songs Played",
        caption="Source: setlist.fm sourced via Concert Analytics"
    ) +
    theme(
        # aspect_ratio = 9 / 16
        # , 
        figure_size=(16,9)
        
        # Backgrounds
        , plot_background=element_rect(fill="#FAF3E0", color=None)
        , panel_background=element_rect(fill="#FAF3E0", color=None)
        , panel_border=element_blank()

        #Margins
        ,plot_margin_top= .03
        ,plot_margin_bottom=.025
        ,plot_margin_right= 0.03
        ,plot_margin_left=0.03

        # Titles & subtitles
        ,plot_title=element_blank()
        #     family=base_family_title+'-Bold'
        #     ,size=30
        #     ,linespacing=1.25
        #     ,weight='bold' 
        #     ,color='#1c1c1c'
        #     ,ha='left'
        #     ,margin={'b': 10 } # spacing below title
        # )

        , plot_subtitle=element_blank()
        #     family=base_family_title
        #     ,size=20
        #     ,color='#777777'
        #     ,ha='left'
        #     ,margin={'b': 30}  # spacing below subtitle
        # )
        
        , plot_caption=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            ,ha='right'
            ,margin={'t': 30}  # spacing above caption
        )

        # Axis
        ,axis_title=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            ,weight='ultralight'
            # ,margin={'t': 5, 'r': 5}  # tweak based on axis
        )
        
        ,axis_text=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            # ,margin={'l':5, 't':20}
        )

        ,axis_title_x=element_blank()
        
        ,axis_ticks=element_line(
            color="#FAF3E0"
        )
        ,panel_grid_major_y = element_line(
            color="#777777"
            ,linetype="dashed"
            ,size=.5
        )

        ,axis_ticks_minor=element_blank()
        ,panel_grid_major_x =element_blank()
        ,panel_grid_minor=element_blank()

        # # Legend
        # ,legend_title=element_blank()
        # ,legend_background=element_blank()
        # ,legend_key=element_blank()
        # ,legend_margin=margins(t=5, b=5)
        # ,legend_box_margin=margins(t=10)

    
    )
)

# g

# g.save(
#     filename="viz/0001-mewithoutYou-Songs-per-show.svg",
#     format="svg",
#     width=16,      # inches
#     height=9,     # adjust to your chart’s shape
#     dpi=300        # high-quality output
# )

## How Many Unique Songs were Played in Total?

In [ ]:
# songs_per_night = 
songs_albums = (
    df.groupby(['track_name','album_name'])['event_set_song_id']
    .nunique()
    .reset_index(name="times_played")
    .sort_values('times_played')
    .reset_index(drop=True)
)

songs_albums['track_name'] = songs_albums['track_name'].apply(
    lambda x: x if len(x) <= 25 else x[:22] + "..."
)

from plotnine import ggplot, aes, geom_col, coord_flip, scale_fill_manual, labs, guides,guide_legend
from plotnine import theme, element_text
import pandas as pd

# Step 1: Make sure song order is preserved
songs_albums_sorted = songs_albums.sort_values('times_played', ascending=True).copy()
songs_albums_sorted['track_name'] = pd.Categorical(
    songs_albums_sorted['track_name'],
    categories=songs_albums_sorted['track_name'],
    ordered=True
)

# Step 3: Create the plot
plot = (
    ggplot(songs_albums_sorted, aes(x='track_name', y='times_played', fill='album_name')) +
    geom_col(show_legend=True) +
    coord_flip() +
    scale_fill_manual(values=album_colors) +
    labs(
        title="Song Frequency on the mewithoutYou [Untitled] Tour",
        subtitle="Each bar shows how many nights a song appeared during the tour",
        x="Song",
        y="Number of Times Played",
        fill=None,
        caption="Source: setlist.fm sourced via Concert Analytics"
    ) +
    theme(
        # aspect_ratio = 9 / 16
        # , 
        figure_size=(12,16)
        
        # Backgrounds
        , plot_background=element_rect(fill="#FAF3E0", color=None)
        , panel_background=element_rect(fill="#FAF3E0", color=None)
        , panel_border=element_blank()

        #Margins
        ,plot_margin_top= .03
        ,plot_margin_bottom=.025
        ,plot_margin_right= 0.03
        ,plot_margin_left=0.03

        # Titles & subtitles
        ,plot_title=element_blank()

        , plot_subtitle=element_blank()
        
        , plot_caption=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            ,ha='right'
            ,margin={'t': 30}  # spacing above caption
        )

        # Axis
        ,axis_title=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            ,weight='ultralight'
            # ,margin={'t': 5, 'r': 5}  # tweak based on axis
        )
        
        ,axis_text=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            # ,margin={'l':5, 't':20}
        )

        ,axis_title_y=element_blank()
        
        ,axis_ticks=element_line(
            color="#FAF3E0"
        )
        ,panel_grid_major_x = element_line(
            color="#777777"
            ,linetype="dashed"
            ,size=.5
        )

        ,axis_ticks_minor=element_blank()
        ,panel_grid_major_y =element_blank()
        ,panel_grid_minor=element_blank()

        # Legend
        ,legend_title=element_blank()
        ,legend_background=element_blank()
        ,legend_key=element_blank()
        ,legend_key_width=10
        , legend_position="top"
        , legend_margin=1
        , legend_direction="vertical"
        # ,legend_margin=margins(t=5, b=5)
        # ,legend_box_margin=margins(t=10)

    
    )  + 
    guides(fill=guide_legend(ncol=2,reverse=True))
)

# plot


# plot.save(
#     filename="viz/0001-mewithoutYou-song-frequency-chart.svg",
#     format="svg",
#     width=12,      # inches
#     height=16,     # adjust to your chart’s shape
#     dpi=300        # high-quality output
# )

## Unique Songs Total by Album

In [ ]:
# unique songs per album

# Count how many unique songs from each album were played
unique_songs_by_album = (
    songs_albums.groupby('album_name')['track_name']
    .nunique()
    .reset_index(name='unique_song_count')
    .sort_values('unique_song_count', ascending=True)
)


# Create a long-form DataFrame where each album is a segment of the bar
stacked_bar_data = unique_songs_by_album.copy()
stacked_bar_data['x'] = ""  # single category for stacking

from pandas.api.types import CategoricalDtype

# Set album_name as a categorical with explicit order
album_order = stacked_bar_data['album_name'].tolist()
album_cat = CategoricalDtype(categories=album_order, ordered=True)
stacked_bar_data['album_name'] = stacked_bar_data['album_name'].astype(album_cat)

# Step 2: Define the color scale
album_colors = {
    "A To B Life": "#A7ECF5",
    "Brother, Sister": "#F986BA",
    "Catch For Us The Foxes": "#3C7DC4",
    "It's All Crazy! It's All False! It's All A Dream! It's Alright": "#D64848",
    "Pale Horses": "#F25C05",
    "Pale Horses: Appendix":"#A7ECF5",
    "Ten Stories": "#5D4E8C",
    "[Untitled]": "#33C27D",
    "[untitled] e.p.": "#F6D357"
}

# stacked_bar_data.head(5)

from plotnine import ggplot, aes, geom_col, coord_flip, scale_fill_manual, labs

stacked_plot = (
    ggplot(stacked_bar_data, aes(x='x', y='unique_song_count', fill='album_name')) +
    geom_col(width=0.5) +  # single stacked bar
    coord_flip() +
    scale_fill_manual(values=album_colors) +
    labs(
        title="Unique Songs Played per Album on the [Untitled] Tour",
        subtitle="Stacked bar showing how many different songs were performed from each album",
        x=None,
        y="Number of Unique Songs",
        caption="Source: setlist.fm sourced via Concert Analytics"
    ) +
    theme(
        # aspect_ratio = 9 / 16
        # , 
        figure_size=(16,9)
        
        # Backgrounds
        , plot_background=element_rect(fill="#FAF3E0", color=None)
        , panel_background=element_rect(fill="#FAF3E0", color=None)
        , panel_border=element_blank()

        #Margins
        ,plot_margin_top= .03
        ,plot_margin_bottom=.025
        ,plot_margin_right= 0.03
        ,plot_margin_left=0.03

        # Titles & subtitles
        ,plot_title=element_blank()

        , plot_subtitle=element_blank()
        
        , plot_caption=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            ,ha='right'
            ,margin={'t': 30}  # spacing above caption
        )

        # Axis
        ,axis_title=element_blank()
        
        ,axis_text=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            # ,margin={'l':5, 't':20}
        )

        ,axis_title_y=element_blank()
        ,axis_title_x=element_blank()
        
        ,axis_ticks=element_line(
            color="#FAF3E0"
        )
        ,panel_grid_major_x = element_line(
            color="#777777"
            ,linetype="dashed"
            ,size=.5
        )

        ,axis_ticks_minor=element_blank()
        ,panel_grid_major_y =element_blank()
        ,panel_grid_minor=element_blank()

        # Legend
        ,legend_title=element_blank()
        ,legend_background=element_blank()
        ,legend_key=element_blank()
        ,legend_key_width=10
        , legend_position="bottom"
        , legend_margin=1
        , legend_direction="vertical"
        # ,legend_margin=margins(t=5, b=5)
        # ,legend_box_margin=margins(t=10)

    
    )  + 
    guides(fill=guide_legend(ncol=4,reverse=True))  # your custom theme with background/colors/fonts
)

# stacked_plot.save(
#     filename="viz/0001-mewithoutYou-Songs-per-Album-chart.svg",
#     format="svg",
#     width=16,      # inches
#     height=9,     # adjust to your chart’s shape
#     dpi=300        # high-quality output
# )

## Most Common Openers

In [ ]:
# songs_per_night = 
opener_songs = (
    df[df.song_index == 1].groupby(['track_name','album_name'])['event_set_song_id']
    .nunique()
    .reset_index(name="times_played")
    .sort_values('times_played')
    .reset_index(drop=True)
)


opener_songs['track_name'] = opener_songs['track_name'].apply(
    lambda x: x if len(x) <= 25 else x[:22] + "..."
)

from plotnine import ggplot, aes, geom_col, coord_flip, scale_fill_manual, labs, guides,guide_legend,element_rect,element_blank,ggplot, aes, geom_col, labs, theme, element_text, element_rect, scale_x_datetime
from plotnine import theme, element_text,scale_y_continuous
import pandas as pd

# Step 1: Make sure song order is preserved
opener_songs_sorted = opener_songs.sort_values('times_played', ascending=True).copy()
opener_songs_sorted['track_name'] = pd.Categorical(
    opener_songs_sorted['track_name'],
    categories=opener_songs_sorted['track_name'],
    ordered=True
)

# Step 3: Create the plot
plot = (
    ggplot(opener_songs_sorted, aes(x='track_name', y='times_played', fill='album_name')) +
    geom_col(show_legend=False) +
    coord_flip() +
    scale_fill_manual(values=album_colors) +
    scale_y_continuous(breaks=range(0, 10, 2)) +
    labs(
        title="Song Frequency on the mewithoutYou [Untitled] Tour",
        subtitle="Each bar shows how many nights a song appeared during the tour",
        x="Song",
        y="Number of Times Played",
        fill=None,
        caption="Source: setlist.fm sourced via Concert Analytics"
    ) +
    theme(
        # aspect_ratio = 9 / 16
        # , 
        figure_size=(12,9)
        
        # Backgrounds
        , plot_background=element_rect(fill="#FAF3E0", color=None)
        , panel_background=element_rect(fill="#FAF3E0", color=None)
        , panel_border=element_blank()

        #Margins
        ,plot_margin_top= .03
        ,plot_margin_bottom=.025
        ,plot_margin_right= 0.03
        ,plot_margin_left=0.03

        # Titles & subtitles
        ,plot_title=element_blank()

        , plot_subtitle=element_blank()
        
        , plot_caption=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            ,ha='right'
            ,margin={'t': 30}  # spacing above caption
        )

        # Axis
        ,axis_title=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            ,weight='ultralight'
            # ,margin={'t': 5, 'r': 5}  # tweak based on axis
        )
        
        ,axis_text=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            # ,margin={'l':5, 't':20}
        )

        ,axis_title_y=element_blank()
        
        ,axis_ticks=element_line(
            color="#FAF3E0"
        )
        ,panel_grid_major_x = element_line(
            color="#777777"
            ,linetype="dashed"
            ,size=.5
        )

        ,axis_ticks_minor=element_blank()
        ,panel_grid_major_y =element_blank()
        ,panel_grid_minor=element_blank()

        # Legend
        ,legend_title=element_blank()
        ,legend_background=element_blank()
        ,legend_key=element_blank()
        ,legend_key_width=10
        , legend_position="top"
        , legend_margin=1
        , legend_direction="vertical"
        # ,legend_margin=margins(t=5, b=5)
        # ,legend_box_margin=margins(t=10)

    
    )  + 
    guides(fill=guide_legend(ncol=2,reverse=True))
)

# plot


plot.save(
    filename="viz/0002-mewithoutYou-Opener_Frequency.svg",
    format="svg",
    width=12,      # inches
    height=9,     # adjust to your chart’s shape
    dpi=300        # high-quality output
)

## Most Common Closer

In [ ]:
# songs_per_night = 
closer_songs = (
    df[df.song_index_reversed == 1].groupby(['track_name','album_name'])['event_set_song_id']
    .nunique()
    .reset_index(name="times_played")
    .sort_values('times_played')
    .reset_index(drop=True)
)


closer_songs['track_name'] = closer_songs['track_name'].apply(
    lambda x: x if len(x) <= 25 else x[:22] + "..."
)

from plotnine import ggplot, aes, geom_col, coord_flip, scale_fill_manual, labs, guides,guide_legend,element_rect,element_blank,ggplot, aes, geom_col, labs, theme, element_text, element_rect, scale_x_datetime
from plotnine import theme, element_text,scale_y_continuous
import pandas as pd

# Step 1: Make sure song order is preserved
closer_songs_sorted = closer_songs.sort_values('times_played', ascending=True).copy()
closer_songs_sorted['track_name'] = pd.Categorical(
    closer_songs_sorted['track_name'],
    categories=closer_songs_sorted['track_name'],
    ordered=True
)

# Step 3: Create the plot
plot = (
    ggplot(closer_songs_sorted, aes(x='track_name', y='times_played', fill='album_name')) +
    geom_col(show_legend=False) +
    coord_flip() +
    scale_fill_manual(values=album_colors) +
    scale_y_continuous(breaks=range(0, 16, 2)) +
    labs(
        title="Song Frequency on the mewithoutYou [Untitled] Tour",
        subtitle="Each bar shows how many nights a song appeared during the tour",
        x="Song",
        y="Number of Times Played",
        fill=None,
        caption="Source: setlist.fm sourced via Concert Analytics"
    ) +
    theme(
        # aspect_ratio = 9 / 16
        # , 
        figure_size=(12,9)
        
        # Backgrounds
        , plot_background=element_rect(fill="#FAF3E0", color=None)
        , panel_background=element_rect(fill="#FAF3E0", color=None)
        , panel_border=element_blank()

        #Margins
        ,plot_margin_top= .03
        ,plot_margin_bottom=.025
        ,plot_margin_right= 0.03
        ,plot_margin_left=0.03

        # Titles & subtitles
        ,plot_title=element_blank()

        , plot_subtitle=element_blank()
        
        , plot_caption=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            ,ha='right'
            ,margin={'t': 30}  # spacing above caption
        )

        # Axis
        ,axis_title=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            ,weight='ultralight'
            # ,margin={'t': 5, 'r': 5}  # tweak based on axis
        )
        
        ,axis_text=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            # ,margin={'l':5, 't':20}
        )

        ,axis_title_y=element_blank()
        
        ,axis_ticks=element_line(
            color="#FAF3E0"
        )
        ,panel_grid_major_x = element_line(
            color="#777777"
            ,linetype="dashed"
            ,size=.5
        )

        ,axis_ticks_minor=element_blank()
        ,panel_grid_major_y =element_blank()
        ,panel_grid_minor=element_blank()

        # Legend
        ,legend_title=element_blank()
        ,legend_background=element_blank()
        ,legend_key=element_blank()
        ,legend_key_width=10
        , legend_position="top"
        , legend_margin=1
        , legend_direction="vertical"
        # ,legend_margin=margins(t=5, b=5)
        # ,legend_box_margin=margins(t=10)

    
    )  + 
    guides(fill=guide_legend(ncol=2,reverse=True))
)

# plot


plot.save(
    filename="viz/0002-mewithoutYou-Closer_Frequency.svg",
    format="svg",
    width=12,      # inches
    height=9,     # adjust to your chart’s shape
    dpi=300        # high-quality output
)

## Most Common Closer of the First Set

In [ ]:
# songs_per_night = 
closer_songs = (
    df.query("song_position_in_set_index_reversed == 1 and set_index == 1")
    .groupby(['track_name','album_name'])['event_set_song_id']
    .nunique()
    .reset_index(name="times_played")
    .sort_values('times_played')
    .reset_index(drop=True)
)


closer_songs['track_name'] = closer_songs['track_name'].apply(
    lambda x: x if len(x) <= 25 else x[:22] + "..."
)

from plotnine import ggplot, aes, geom_col, coord_flip, scale_fill_manual, labs, guides,guide_legend,element_rect,element_blank,ggplot, aes, geom_col, labs, theme, element_text, element_rect, scale_x_datetime
from plotnine import theme, element_text,scale_y_continuous
import pandas as pd

# Step 1: Make sure song order is preserved
closer_songs_sorted = closer_songs.sort_values('times_played', ascending=True).copy()
closer_songs_sorted['track_name'] = pd.Categorical(
    closer_songs_sorted['track_name'],
    categories=closer_songs_sorted['track_name'],
    ordered=True
)

# Step 3: Create the plot
plot = (
    ggplot(closer_songs_sorted, aes(x='track_name', y='times_played', fill='album_name')) +
    geom_col(show_legend=False) +
    coord_flip() +
    scale_fill_manual(values=album_colors) +
    scale_y_continuous(breaks=range(0, 18, 2)) +
    labs(
        title="Song Frequency on the mewithoutYou [Untitled] Tour",
        subtitle="Each bar shows how many nights a song appeared during the tour",
        x="Song",
        y="Number of Times Played",
        fill=None,
        caption="Source: setlist.fm sourced via Concert Analytics"
    ) +
    theme(
        # aspect_ratio = 9 / 16
        # , 
        figure_size=(12,9)
        
        # Backgrounds
        , plot_background=element_rect(fill="#FAF3E0", color=None)
        , panel_background=element_rect(fill="#FAF3E0", color=None)
        , panel_border=element_blank()

        #Margins
        ,plot_margin_top= .03
        ,plot_margin_bottom=.025
        ,plot_margin_right= 0.03
        ,plot_margin_left=0.03

        # Titles & subtitles
        ,plot_title=element_blank()

        , plot_subtitle=element_blank()
        
        , plot_caption=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            ,ha='right'
            ,margin={'t': 30}  # spacing above caption
        )

        # Axis
        ,axis_title=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            ,weight='ultralight'
            # ,margin={'t': 5, 'r': 5}  # tweak based on axis
        )
        
        ,axis_text=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            # ,margin={'l':5, 't':20}
        )

        ,axis_title_y=element_blank()
        
        ,axis_ticks=element_line(
            color="#FAF3E0"
        )
        ,panel_grid_major_x = element_line(
            color="#777777"
            ,linetype="dashed"
            ,size=.5
        )

        ,axis_ticks_minor=element_blank()
        ,panel_grid_major_y =element_blank()
        ,panel_grid_minor=element_blank()

        # Legend
        ,legend_title=element_blank()
        ,legend_background=element_blank()
        ,legend_key=element_blank()
        ,legend_key_width=10
        , legend_position="top"
        , legend_margin=1
        , legend_direction="vertical"
        # ,legend_margin=margins(t=5, b=5)
        # ,legend_box_margin=margins(t=10)

    
    )  + 
    guides(fill=guide_legend(ncol=2,reverse=True))
)

# plot


plot.save(
    filename="viz/0002-mewithoutYou-First_Set_Closer_Frequency.svg",
    format="svg",
    width=12,      # inches
    height=9,     # adjust to your chart’s shape
    dpi=300        # high-quality output
)


## Most Common Opener of the last set

In [ ]:
# songs_per_night = 
closer_songs = (
    df.query("song_position_in_set_index == 1 and set_index_reversed == 1")
    .groupby(['track_name','album_name'])['event_set_song_id']
    .nunique()
    .reset_index(name="times_played")
    .sort_values('times_played')
    .reset_index(drop=True)
)


closer_songs['track_name'] = closer_songs['track_name'].apply(
    lambda x: x if len(x) <= 25 else x[:22] + "..."
)

from plotnine import ggplot, aes, geom_col, coord_flip, scale_fill_manual, labs, guides,guide_legend,element_rect,element_blank,ggplot, aes, geom_col, labs, theme, element_text, element_rect, scale_x_datetime
from plotnine import theme, element_text,scale_y_continuous
import pandas as pd

# Step 1: Make sure song order is preserved
closer_songs_sorted = closer_songs.sort_values('times_played', ascending=True).copy()
closer_songs_sorted['track_name'] = pd.Categorical(
    closer_songs_sorted['track_name'],
    categories=closer_songs_sorted['track_name'],
    ordered=True
)

# Step 3: Create the plot
plot = (
    ggplot(closer_songs_sorted, aes(x='track_name', y='times_played', fill='album_name')) +
    geom_col(show_legend=False) +
    coord_flip() +
    scale_fill_manual(values=album_colors) +
    scale_y_continuous(breaks=range(0, 18, 2)) +
    labs(
        title="Song Frequency on the mewithoutYou [Untitled] Tour",
        subtitle="Each bar shows how many nights a song appeared during the tour",
        x="Song",
        y="Number of Times Played",
        fill=None,
        caption="Source: setlist.fm sourced via Concert Analytics"
    ) +
    theme(
        # aspect_ratio = 9 / 16
        # , 
        figure_size=(12,9)
        
        # Backgrounds
        , plot_background=element_rect(fill="#FAF3E0", color=None)
        , panel_background=element_rect(fill="#FAF3E0", color=None)
        , panel_border=element_blank()

        #Margins
        ,plot_margin_top= .03
        ,plot_margin_bottom=.025
        ,plot_margin_right= 0.03
        ,plot_margin_left=0.03

        # Titles & subtitles
        ,plot_title=element_blank()

        , plot_subtitle=element_blank()
        
        , plot_caption=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            ,ha='right'
            ,margin={'t': 30}  # spacing above caption
        )

        # Axis
        ,axis_title=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            ,weight='ultralight'
            # ,margin={'t': 5, 'r': 5}  # tweak based on axis
        )
        
        ,axis_text=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            # ,margin={'l':5, 't':20}
        )

        ,axis_title_y=element_blank()
        
        ,axis_ticks=element_line(
            color="#FAF3E0"
        )
        ,panel_grid_major_x = element_line(
            color="#777777"
            ,linetype="dashed"
            ,size=.5
        )

        ,axis_ticks_minor=element_blank()
        ,panel_grid_major_y =element_blank()
        ,panel_grid_minor=element_blank()

        # Legend
        ,legend_title=element_blank()
        ,legend_background=element_blank()
        ,legend_key=element_blank()
        ,legend_key_width=10
        , legend_position="top"
        , legend_margin=1
        , legend_direction="vertical"
        # ,legend_margin=margins(t=5, b=5)
        # ,legend_box_margin=margins(t=10)

    
    )  + 
    guides(fill=guide_legend(ncol=2,reverse=True))
)

# plot


plot.save(
    filename="viz/0002-mewithoutYou-Encore_Set_Opener_Frequency.svg",
    format="svg",
    width=12,      # inches
    height=9,     # adjust to your chart’s shape
    dpi=300        # high-quality output
)


## The average [Untitled] Set list 

In [ ]:
avg_setlist = (
    df.groupby(['track_name','album_name'])
    .agg(
        min_position=('song_index', 'min')
        ,p25=('song_index', lambda x: x.quantile(0.35))
        ,avg_position=('song_index', 'mean')
        ,median_position=('song_index', 'median')
        ,p75=('song_index', lambda x: x.quantile(0.65))
        ,max_position=('song_index', 'max')
        ,times_played=('song_index', 'count')
    )
    .reset_index()
    .sort_values('median_position', ascending=False)
    .reset_index(drop=True)
)

# Assume you already have median_position in summary
# Now count how many times each song appears at exactly its median
counts_at_median = (
    df.merge(avg_setlist[['track_name', 'median_position']], on='track_name')
    .assign(is_at_median=lambda d: d['song_index'] == d['median_position'])
    .groupby('track_name')['is_at_median']
    .sum()
    .reset_index(name='median_count')
)

# Merge back into summary
avg_setlist = avg_setlist.merge(counts_at_median, on='track_name', how='left')
avg_setlist['median_count'] = avg_setlist['median_count'].fillna(1)

from pandas.api.types import CategoricalDtype

cat_type = CategoricalDtype(categories=avg_setlist['track_name'], ordered=True)
avg_setlist['track_name'] = avg_setlist['track_name'].astype(cat_type)

avg_setlist['track_name'] = avg_setlist['track_name'].apply(
    lambda x: x if len(x) <= 25 else x[:22] + "..."
)

# avg_setlist.head(60)

from plotnine import ggplot, aes, geom_segment, geom_point, theme_minimal, labs, theme, element_text,scale_color_manual,scale_size_continuous

# Build the plot
plot = (
    ggplot(avg_setlist, aes(y='track_name'))
    + geom_point(
        aes(x='min_position', color='album_name'),
        size=1.5
        ,show_legend=False
        ,alpha=0.3
    )
    + geom_point(
        aes(x='max_position', color='album_name'),
        size=1.5
        ,show_legend=False
        ,alpha=0.3
    )
    + geom_segment(
        aes(x='p25', xend='p75', y='track_name', yend='track_name', color='album_name'),
        size=2
        ,show_legend=False
        ,alpha=0.5
    )
    + geom_point(
        aes(
            x='median_position'
            , color='album_name'
            , size='median_count'
            )
        ,show_legend=True
    )
    + scale_size_continuous(range=[1, 5])
    + scale_color_manual(values=album_colors) 
    + labs(
        title='Song Positions Across Setlists',
        subtitle='Each line shows min to max position for a song, dot shows average',
        x='Setlist Position',
        y='Song',
        color='Album'
        ,caption="Source: setlist.fm sourced via Concert Analytics"
    )
    +
    theme(
        # aspect_ratio = 9 / 16
        # , 
        figure_size=(12,19)
        
        # Backgrounds
        , plot_background=element_rect(fill="#FAF3E0", color=None)
        , panel_background=element_rect(fill="#FAF3E0", color=None)
        , panel_border=element_blank()

        #Margins
        ,plot_margin_top= .03
        ,plot_margin_bottom=.025
        ,plot_margin_right= 0.03
        ,plot_margin_left=0.03

        # Titles & subtitles
        ,plot_title=element_blank()

        , plot_subtitle=element_blank()
        
        , plot_caption=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            ,ha='right'
            ,margin={'t': 30}  # spacing above caption
        )

        # Axis
        ,axis_title=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            ,weight='ultralight'
            # ,margin={'t': 5, 'r': 5}  # tweak based on axis
        )
        
        ,axis_text=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            # ,margin={'l':5, 't':20}
        )

        ,axis_title_y=element_blank()
        
        ,axis_ticks=element_line(
            color="#FAF3E0"
        )
        ,panel_grid_major_x = element_line(
            color="#777777"
            ,linetype="dashed"
            ,size=.5
        )

        ,axis_ticks_minor=element_blank()
        ,panel_grid_major_y =element_blank()
        ,panel_grid_minor=element_blank()

        # Legend
        ,legend_title=element_blank()
        ,legend_background=element_blank()
        ,legend_key=element_blank()
        ,legend_key_width=10
        , legend_position="top"
        , legend_margin=1
        , legend_direction="vertical"
        # ,legend_margin=margins(t=5, b=5)
        # ,legend_box_margin=margins(t=10)

    
    )  + 
    guides(fill=guide_legend(ncol=2,reverse=True))
)

# plot

plot.save(
    filename="viz/0002-mewithoutYou-Setlist_Position_Spread.svg",
    format="svg",
    width=12,      # inches
    height=19,     # adjust to your chart’s shape
    dpi=300        # high-quality output
)

# avg_setlist.head(60)


## Album Representation per Show

## How popularity moves within a set

### Time as a dot

In [ ]:

# Table I want is
# event_id, event_position_relative, event_set_song_id, popularity

event_position_time_popularity = df[['event_id'
                                     ,'event_set_song_id'
                                     ,'track_name'
                                     ,'track_popularity'
                                     ,'song_index'
                                     ,'event_total_songs'
                                     ,'track_duration_seconds'
                                     ,'encore_flag'
                                     ,'song_position_in_set_index'
                                     ]].copy()

event_position_time_popularity['event_position_relative'] = event_position_time_popularity['song_index'] / event_position_time_popularity['event_total_songs']

# Step 2: Get total duration per event
event_position_time_popularity['track_duration_min'] = event_position_time_popularity['track_duration_seconds'] / 60

# Step 3: Cumulative time in the event
event_position_time_popularity = (
    event_position_time_popularity
    .sort_values(['event_id', 'event_position_relative'])  # or position index if you have it
    .assign(cum_time=lambda event_position_time_popularity: event_position_time_popularity.groupby('event_id')['track_duration_min'].cumsum())
)

# df.head()

# Step 4: Normalize by total event duration
total_durations = (
    event_position_time_popularity
    .groupby('event_id')['track_duration_min']
    .sum()
    .rename('total_duration_min')
    .reset_index()
)

event_position_time_popularity = event_position_time_popularity.merge(total_durations, on='event_id')
event_position_time_popularity['time_relative'] = event_position_time_popularity['cum_time'] / event_position_time_popularity['total_duration_min']

average_encore_position = (
    event_position_time_popularity.query("encore_flag == True and song_position_in_set_index == 1")['time_relative']
    .mean()
)

event_position_time_popularity.head()


from plotnine import *
import pandas as pd
base_family_axis="Courier New"
base_family_title="Helvetica"

# Calculate the average popularity at each relative position
bins = 20
event_position_time_popularity['bin'] = pd.cut(
    event_position_time_popularity['time_relative'],
    bins=bins,
    labels=False,
    include_lowest=True
)
avg_df = (
    event_position_time_popularity
    .groupby('bin', as_index=False)
    .agg({'track_popularity': 'mean'
          ,"event_set_song_id": "count"
          })
)
avg_df['x'] = (avg_df['bin'] + .75) / bins

# Build the chart
chart = (
    ggplot()
    # Individual event lines with alpha for opacity
    + geom_line(
        data=event_position_time_popularity,
        mapping=aes(x="time_relative", y="track_popularity", group="event_id"),
        color="#777777",
        alpha=0.25
    )
    # Average line in bold
    + geom_line(
        data=avg_df,
        mapping=aes(x="x", y="track_popularity"),
        color="#33C27D",
        size=2
    )
    + geom_vline(
    xintercept=average_encore_position,
    linetype="dashed",
    color="#F25C05",
    size=1,
    alpha=0.8
    )
    + annotate(
        'text',
        x=average_encore_position - 0.05,
        y=event_position_time_popularity['track_popularity'].max() + 1,
        label='Encore Avg.',
        size=10,
        color="#F25C05",
        family=base_family_axis
    )
    + theme(
        # aspect_ratio = 9 / 16
        # , 
        figure_size=(16,9)
        
        # Backgrounds
        , plot_background=element_rect(fill="#FAF3E0", color=None)
        , panel_background=element_rect(fill="#FAF3E0", color=None)
        , panel_border=element_blank()

        #Margins
        ,plot_margin_top= .03
        ,plot_margin_bottom=.025
        ,plot_margin_right= 0.03
        ,plot_margin_left=0.03

        # Titles & subtitles
        ,plot_title=element_blank()
        #     family=base_family_title+'-Bold'
        #     ,size=30
        #     ,linespacing=1.25
        #     ,weight='bold' 
        #     ,color='#1c1c1c'
        #     ,ha='left'
        #     ,margin={'b': 10 } # spacing below title
        # )

        , plot_subtitle=element_blank()
        #     family=base_family_title
        #     ,size=20
        #     ,color='#777777'
        #     ,ha='left'
        #     ,margin={'b': 30}  # spacing below subtitle
        # )
        
        , plot_caption=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            ,ha='right'
            ,margin={'t': 30}  # spacing above caption
        )

        # Axis
        ,axis_title=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            ,weight='ultralight'
            # ,margin={'t': 5, 'r': 5}  # tweak based on axis
        )
        
        ,axis_text=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            # ,margin={'l':5, 't':20}
        )

        ,axis_title_x=element_blank()
        
        ,axis_ticks=element_line(
            color="#FAF3E0"
        )
        ,panel_grid_major_y = element_blank()
        # element_line(
        #     color="#777777"
        #     ,linetype="dashed"
        #     ,size=.5
        # )

        ,axis_ticks_minor=element_blank()
        ,panel_grid_major_x =element_blank()
        ,panel_grid_minor=element_blank()
        # ,panel_grid_major_y=element_blank()

        # # Legend
        # ,legend_title=element_blank()
        # ,legend_background=element_blank()
        # ,legend_key=element_blank()
        # ,legend_margin=margins(t=5, b=5)
        # ,legend_box_margin=margins(t=10)

    
    )
)

chart
# avg_df.head(100)

### Time as a segment

In [ ]:

# Table I want is
# event_id, event_position_relative, event_set_song_id, popularity

event_position_time_popularity = df[['event_id'
                                     ,'event_set_song_id'
                                     ,'track_name'
                                     ,'track_popularity'
                                     ,'song_index'
                                     ,'event_total_songs'
                                     ,'track_duration_seconds'
                                     ,'encore_flag'
                                     ,'song_position_in_set_index'
                                     ]].copy()

event_position_time_popularity['event_position_relative'] = event_position_time_popularity['song_index'] / event_position_time_popularity['event_total_songs']

# Step 2: Get total duration per event
event_position_time_popularity['track_duration_min'] = event_position_time_popularity['track_duration_seconds'] / 60

# Step 3: Cumulative time in the event
event_position_time_popularity = (
    event_position_time_popularity
    .sort_values(['event_id', 'event_position_relative'])  # or position index if you have it
    .assign(cum_time=lambda event_position_time_popularity: event_position_time_popularity.groupby('event_id')['track_duration_min'].cumsum())
)

# df.head()

# Step 4: Normalize by total event duration
total_durations = (
    event_position_time_popularity
    .groupby('event_id')['track_duration_min']
    .sum()
    .rename('total_duration_min')
    .reset_index()
)

step_df = event_position_time_popularity.merge(total_durations, on='event_id')

step_df['start_time'] = step_df['cum_time'] - step_df['track_duration_min']
step_df['x_start'] = step_df['start_time'] / step_df['total_duration_min']
step_df['x_end'] = (step_df['cum_time']-.001) / step_df['total_duration_min']


# Calculate the average popularity at each relative position
import numpy as np

# Create bin edges and labels
bin_size = 40
bin_edges = np.linspace(0, 1, bin_size+1)  # 21 edges = 20 bins
bin_labels = [(bin_edges[i], bin_edges[i+1]) for i in range(len(bin_edges)-1)]

rows = []
for i, (start, end) in enumerate(bin_labels):
    # Find all songs active during this bin
    active_songs = step_df[(step_df['x_start'] < end) & (step_df['x_end'] > start)]
    
    if not active_songs.empty:
        avg_pop = active_songs['track_popularity'].mean()
        rows.append({
            'event_id': 'average'
            ,'bin': i
            ,'track_popularity': avg_pop
            ,'x_start': start
            ,'x_end': end-.00001
            ,'count': len(active_songs)
        })

avg_step_df = pd.DataFrame(rows)

# Build Step Lines
def build_steps(df):
    starts = df[['event_id', 'x_start', 'track_popularity']].copy()
    ends = df[['event_id', 'x_end', 'track_popularity']].copy()
    starts.columns = ['event_id', 'x', 'track_popularity']
    ends.columns = ['event_id', 'x', 'track_popularity']
    return pd.concat([starts, ends]).sort_values(['event_id', 'x'])

step_lines = build_steps(step_df)
average_step_lines = build_steps(avg_step_df)

# Vertical Lines
average_encore_position = (
    step_df.query("encore_flag == True and song_position_in_set_index == 1")['x_start']
    .mean()
)

thirty_min_in_position = 30 / total_durations['total_duration_min'].mean() 


from plotnine import *
import pandas as pd
base_family_axis="Courier New"
base_family_title="Helvetica"



# Build the chart
chart = (
    ggplot()
    # Individual event lines with alpha for opacity
    + geom_line(
        data=step_lines,
        mapping=aes(x='x', y='track_popularity', group='event_id'),
        alpha=0.2,
        color="#777777"
    )
    # Average line in bold
    + geom_line(
        data=average_step_lines,
        mapping=aes(x="x", y="track_popularity"),
        color="#33C27D",
        size=1.5
    )
    + geom_vline(
        xintercept=average_encore_position,
        linetype="dashed",
        color="#F25C05",
        size=1,
        alpha=0.8
    )
    + annotate(
        'text',
        x=average_encore_position - 0.05,
        y=event_position_time_popularity['track_popularity'].max() + 1,
        label='Encore Avg.',
        size=10,
        color="#F25C05",
        family=base_family_axis
    )
    + geom_vline(
        xintercept=thirty_min_in_position,
        linetype="dashed",
        color="#F25C05",
        size=1,
        alpha=0.8
    )
    + annotate(
        'text',
        x=thirty_min_in_position - 0.05,
        y=event_position_time_popularity['track_popularity'].max() + 1,
        label='30 Mins In',
        size=10,
        color="#F25C05",
        family=base_family_axis
    )
    + theme(
        # aspect_ratio = 9 / 16
        # , 
        figure_size=(16,9)
        
        # Backgrounds
        , plot_background=element_rect(fill="#FAF3E0", color=None)
        , panel_background=element_rect(fill="#FAF3E0", color=None)
        , panel_border=element_blank()

        #Margins
        ,plot_margin_top= .03
        ,plot_margin_bottom=.025
        ,plot_margin_right= 0.03
        ,plot_margin_left=0.03

        # Titles & subtitles
        ,plot_title=element_blank()
        #     family=base_family_title+'-Bold'
        #     ,size=30
        #     ,linespacing=1.25
        #     ,weight='bold' 
        #     ,color='#1c1c1c'
        #     ,ha='left'
        #     ,margin={'b': 10 } # spacing below title
        # )

        , plot_subtitle=element_blank()
        #     family=base_family_title
        #     ,size=20
        #     ,color='#777777'
        #     ,ha='left'
        #     ,margin={'b': 30}  # spacing below subtitle
        # )
        
        , plot_caption=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            ,ha='right'
            ,margin={'t': 30}  # spacing above caption
        )

        # Axis
        ,axis_title=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            ,weight='ultralight'
            # ,margin={'t': 5, 'r': 5}  # tweak based on axis
        )
        
        ,axis_text=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            # ,margin={'l':5, 't':20}
        )

        ,axis_title_x=element_blank()
        
        ,axis_ticks=element_line(
            color="#FAF3E0"
        )
        ,panel_grid_major_y = element_blank()
        # element_line(
        #     color="#777777"
        #     ,linetype="dashed"
        #     ,size=.5
        # )

        ,axis_ticks_minor=element_blank()
        ,panel_grid_major_x =element_blank()
        ,panel_grid_minor=element_blank()
        # ,panel_grid_major_y=element_blank()

        # # Legend
        # ,legend_title=element_blank()
        # ,legend_background=element_blank()
        # ,legend_key=element_blank()
        # ,legend_margin=margins(t=5, b=5)
        # ,legend_box_margin=margins(t=10)

    
    )
)

chart
# avg_step_df.head(100)
# df.head(30)

## City that got the worst or best set list

## How popularity moved for the tour (avg. Popularity per show)

## Legend Export

In [ ]:
from plotnine import ggplot, aes, geom_point, scale_color_manual, scale_size_continuous, theme_void, theme

# Make a dummy dataframe
import pandas as pd

legend_data = pd.DataFrame({
    'album_name': list(album_colors.keys()),  # your album list
    'dummy_x': [1]*len(album_colors),
    'dummy_y': list(range(len(album_colors)))
})

# Dummy plot for album color legend
legend_plot = (
    ggplot(legend_data, aes(x='dummy_x', y='dummy_y', color='album_name'))
    + geom_point(size=5)  # Bigger dots so legend is easier to read
    + scale_color_manual(values=album_colors)
    + theme_void()
    + theme(
        legend_position='right',
        legend_direction='vertical',
        legend_title=element_blank()
    )
)

legend_plot.save('viz/legend_albums.svg', dpi=300)

In [ ]:
size_data = pd.DataFrame({
    'size_var': [1, 5, 10, 15]
})

size_plot = (
    ggplot(size_data, aes(x='size_var', y='size_var', size='size_var'))
    + geom_point(color='black')
    + scale_size_continuous(range=[1, 6])
    + theme_void()
    + theme(
        legend_position='right',
        legend_direction='vertical',
        legend_title=element_blank()
    )
)

size_plot.save('viz/legend_sizes.svg', dpi=300)

In [ ]:

# Table I want is
# event_id, event_position_relative, event_set_song_id, popularity

df['event_position_relative'] = df['song_index'] / df['event_total_songs']

event_position_popularity = df[['event_id','event_position_relative','event_set_song_id','track_name','track_popularity']].copy()



# event_position_popularity.head()

from plotnine import *
import pandas as pd
base_family_axis="Courier New"
base_family_title="Helvetica"

# Calculate the average popularity at each relative position
bins = 20
event_position_popularity['bin'] = pd.cut(
    event_position_popularity['event_position_relative'],
    bins=bins,
    labels=False,
    include_lowest=True
)
avg_df = (
    event_position_popularity
    .groupby('bin', as_index=False)
    .agg({'track_popularity': 'mean'
          ,"event_set_song_id": "count"
          })
)
avg_df['x'] = (avg_df['bin'] + 1) / bins

# Build the chart
chart = (
    ggplot()
    # Individual event lines with alpha for opacity
    + geom_line(
        data=event_position_popularity,
        mapping=aes(x="event_position_relative", y="track_popularity", group="event_id"),
        color="#777777",
        alpha=0.5
    )
    # Average line in bold
    + geom_line(
        data=avg_df,
        mapping=aes(x="x", y="track_popularity"),
        color="#33C27D",
        size=2
    )
    + theme(
        # aspect_ratio = 9 / 16
        # , 
        figure_size=(16,9)
        
        # Backgrounds
        , plot_background=element_rect(fill="#FAF3E0", color=None)
        , panel_background=element_rect(fill="#FAF3E0", color=None)
        , panel_border=element_blank()

        #Margins
        ,plot_margin_top= .03
        ,plot_margin_bottom=.025
        ,plot_margin_right= 0.03
        ,plot_margin_left=0.03

        # Titles & subtitles
        ,plot_title=element_blank()
        #     family=base_family_title+'-Bold'
        #     ,size=30
        #     ,linespacing=1.25
        #     ,weight='bold' 
        #     ,color='#1c1c1c'
        #     ,ha='left'
        #     ,margin={'b': 10 } # spacing below title
        # )

        , plot_subtitle=element_blank()
        #     family=base_family_title
        #     ,size=20
        #     ,color='#777777'
        #     ,ha='left'
        #     ,margin={'b': 30}  # spacing below subtitle
        # )
        
        , plot_caption=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            ,ha='right'
            ,margin={'t': 30}  # spacing above caption
        )

        # Axis
        ,axis_title=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            ,weight='ultralight'
            # ,margin={'t': 5, 'r': 5}  # tweak based on axis
        )
        
        ,axis_text=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            # ,margin={'l':5, 't':20}
        )

        ,axis_title_x=element_blank()
        
        ,axis_ticks=element_line(
            color="#FAF3E0"
        )
        ,panel_grid_major_y = element_blank()
        # element_line(
        #     color="#777777"
        #     ,linetype="dashed"
        #     ,size=.5
        # )

        ,axis_ticks_minor=element_blank()
        ,panel_grid_major_x =element_blank()
        ,panel_grid_minor=element_blank()
        # ,panel_grid_major_y=element_blank()

        # # Legend
        # ,legend_title=element_blank()
        # ,legend_background=element_blank()
        # ,legend_key=element_blank()
        # ,legend_margin=margins(t=5, b=5)
        # ,legend_box_margin=margins(t=10)

    
    )
)

chart
# avg_df.head(100)